### **Código de árbol B (B-Tree)**

En este cuaderno, hemos proporcionado el código para el árbol B (B-Tree) junto con una breve descripción de cómo implementar las distintas operaciones. La explicación que sigue se basa en esta excelente revisión:

> Douglas Comer, <a href="https://dl.acm.org/doi/pdf/10.1145/356770.356776"> Ubiquitous B-Tree. </a> ACM Comput. Surv. 11, 2 (junio de 1979), 121–137. 

Se recomienda revisar esta publicación: hace un excelente trabajo explicando las técnicas a un nivel alto con ejemplos.


La implementación ha sido construida en tres etapas:  
- `BTreeBaseNode`: implementa todos los campos necesarios para un nodo de Árbol B y algunas funciones utilitarias importantes.  
- `BTreeWithNodeInsert`: implementa la función para insertar una clave en el Árbol B.  
- `BTreeNode`: añade la función para eliminar una clave sobre la base de `BTreeWithInsert`.

Cada clase hereda de la anterior.

#### **Estructura de un nodo de árbol B**

Cada nodo de un árbol B tendrá los siguientes campos:

- `d`: el valor del parámetro $d$ del árbol B. Este debe ser el mismo para todos los nodos del árbol.  
- `keys`: una lista de claves ordenadas en orden ascendente. La longitud de `keys` debe estar entre 0 y $2d$ para el nodo raíz, y entre $d$ y $2d$ para los nodos internos.  
- `pointers`: lista de punteros a nodos hijos.  
  - Si el nodo es una hoja, la longitud de `pointers` será $0$.  
  - Si el nodo no es una hoja, la longitud de `pointers` será uno más que la longitud de `keys`.  
- `is_root`: valor booleano que indica si el nodo es la raíz.  
- `parent`: para un nodo raíz esto será `None`, de lo contrario será una tupla de la forma `(parent_node, parent_idx)`, donde:  
  - `parent_node` es un puntero al nodo padre del nodo actual.  
  - `parent_idx` es el índice del nodo actual en la lista `parent_node.pointers`.  
  - En otras palabras, `parent_node.pointers[parent_idx] == self`.  

Para una clave `keys[i]`, su hijo izquierdo se encuentra en `pointers[i]`, que apuntará a un nodo cuyas claves son $< \text{keys}[i]$; y su hijo derecho está en `pointers[i+1]`, que apuntará a un nodo cuyas claves son $> \text{keys}[i]$ (pero menores que $\text{keys}[i+1]$, si esta existe).

#### **Implementación de búsqueda (Find)**

Tenemos una implementación de la función `find` para el árbol B, la cual utiliza la función `find_key_internal`.  
El método `find_key_internal` busca el nodo que contiene una clave y el índice dentro del nodo, o devuelve `None` si la clave no está en el árbol.

- Comenzando desde la raíz, busca la clave en el nodo actual.  
  - Si encuentra la clave, la devuelve inmediatamente.  
- De lo contrario, obtiene el nodo hijo cuyo subárbol contendría la clave (si existe en el árbol), y llama recursivamente a `find_key_internal` en ese nodo hijo.

In [ ]:
import os
os.environ["PATH"] = r"C:\Program Files\Graphviz\bin;" + os.environ["PATH"]


In [ ]:
class BTreeNodeBase(object):
    def __init__(self, keys = [], ptrs = [], is_root=False, d = 10):
        # Cada nodo interno debe contener al menos d claves y como máximo 2d claves
        # La raíz puede contener entre 0 claves (si todo el árbol está vacío) y 2d claves
        self.keys = list(keys)  # las claves
        self.d = d  # el valor de d
        self.pointers = list(ptrs)  # los punteros
        self.is_root = is_root
        self.parent = None  # ya sea None o una tupla (nodo_padre, idx) tal que nodo_padre.pointers[idx] == self

    def is_leaf(self):
        """Retorna True si el nodo es una hoja, de lo contrario retorna False"""
        return len(self.pointers) == 0

    def set_parent(self, parent_node, idx):
        assert parent_node != None
        assert 0 <= idx and idx < len(parent_node.pointers)
        assert parent_node.pointers[idx] == self
        self.parent = (parent_node, idx)

    def find_key_internal(self, k):
        """find_key_internal para una clave k retorna la referencia al nodo y el índice en el arreglo de claves si se encuentra.
           De lo contrario, retorna None
        """
        n = len(self.keys)
        if n == 0:  # nodo vacío, retornar False
            return None
        # encontrar el primer índice i tal que self.keys[i] >= k
        i = 0
        while i < n and self.keys[i] < k:
            i = i + 1
        if i < n and self.keys[i] == k:
            return (self, i)  # clave encontrada
        else:
            if self.is_leaf():  # si estamos en una hoja, la clave no se encuentra
                return None
            else:  # de lo contrario, buscamos recursivamente en el nodo hijo apropiado
                return self.pointers[i].find_key_internal(k)

    def find_key(self, k):
        """Función find_key que debe llamarse en el nodo raíz"""
        assert self.is_root
        res = self.find_key_internal(k)  # llamar al método find_key_internal
        return True if res != None else False

    def find_successor(self, idx):
        """find_successor: recorrer el nodo a la derecha de idx
           y luego seguir recorriendo los hijos izquierdos hasta llegar a una hoja"""
        assert idx >= 0 and idx < len(self.keys)  # verificar que el índice sea válido
        assert not self.is_leaf()  # no llamar a esta función en una hoja
        child = self.pointers[idx+1]  # obtener el hijo derecho
        while not child.is_leaf():
            child = child.pointers[0]  # ir hacia la izquierda
        assert child.is_leaf()  # se llegó a la hoja más a la izquierda
        return (child.keys[0], child)

    def __str__(self):
        return str(self.keys)

    def make_networkx_graph(self, G, node_id, parent_id, label_dict):
        """Esto es para visualización. Añade los detalles del nodo incluyendo sus aristas salientes
           al grafo G para poder dibujar el árbol."""
        node_label = str(self.keys)
        if self.parent != None:
            node_label = "C"+str(self.parent[1]) + ": " + node_label
        else:
            node_label = "R: "+ node_label
        G.add_node(node_id, label=node_label)
        label_dict[node_id] = node_label
        if parent_id >= 0:
            G.add_edge(parent_id, node_id)
        n = len(self.pointers)
        new_id = node_id+1
        for i in range(n):
            new_id = self.pointers[i].make_networkx_graph(G, new_id, node_id, label_dict)
        return new_id + 1

    def rep_ok(self):
        """Verifica si el Árbol B respeta las propiedades."""
        n = len(self.keys)
        p = len(self.pointers)
        d = self.d
        # el nodo es una hoja sin punteros o debe tener un puntero más que el número de claves
        assert p == 0 or p == n + 1, f'El nodo tiene {n} claves pero {p} punteros'
        # verificar si las claves están en orden ascendente
        for i in range(1, n):
            assert self.keys[i] > self.keys[i-1], f'Las claves {keys[i-1]} y {keys[i]} no están en orden ascendente'
        if self.is_root:
            assert self.parent == None  # la raíz no tiene padre
            assert 0 <= n and n <= 2 * d  # número de claves de la raíz debe estar entre [0, 2d]
            self.check_height_properties()  # verificar que todas las rutas desde la raíz a las hojas tengan la misma longitud
        else:
            assert self.parent != None  # un nodo que no es la raíz debe tener un padre
            assert d <= n and n <= 2 * d  # número de claves debe estar entre [d, 2d]
        if p >= 1:
            for (j, child_node) in enumerate(self.pointers):  # para cada nodo hijo
                assert child_node.parent == (self, j)  # ¿el puntero al padre en el hijo es correcto?
                assert child_node.d == self.d
                assert not child_node.is_root
                child_node.rep_ok()  # verificar recursivamente si el nodo hijo respeta las propiedades

    def check_height_properties(self):
        """Verificar que la altura de todos los nodos hijos sea la misma y retornar la altura del nodo actual"""
        if self.is_leaf():
            return 0
        else:
            depths = [child.check_height_properties() for child in self.pointers]
            d0 = depths[0]
            assert all(di == d0 for di in depths), f'El nodo con claves {self.keys} tiene alturas inconsistentes'
            return 1 + d0

    def create_new_instance(self, keys, ptrs, is_root, d):
        """Necesitamos esto para construir una nueva instancia porque decidimos dividir la implementación en tres clases diferentes"""
        return BTreeNodeBase(keys, ptrs, is_root, d)


#### **Insertar una clave en el B-Tree**

Vamos a implementar la función `insert`, que inserta una clave `new_key` en el árbol. La lógica principal se implementa en la función `insert_helper`.

<div class="alert alert-block" style="background-color:lightcyan; border-color:black white black white">
  Asumiremos que la clave a insertar no está presente en el árbol, aunque nuestra implementación también maneja este caso. Ten en cuenta que, en un árbol de búsqueda, una clave puede aparecer como máximo una vez.
</div>

La función `insert_helper` consta de dos fases:
- Primero, desciende desde la raíz hasta una hoja como si buscara la clave `new_key` que queremos insertar.
- Luego, inserta `new_key` en la hoja y asciende de nuevo hacia la raíz para corregir los nodos que puedan haberse llenado como resultado de la inserción.

##### **(A) Búsqueda de la hoja que contendrá la nueva clave**

La primera fase desciende desde la raíz hasta una hoja que contendrá la nueva clave a insertar.
- En cada nodo no-hoja, buscamos un índice `i` tal que `node.keys[i] > new_key`. Si no se encuentra tal `i`, la `new_key` se insertará en la rama más a la derecha del árbol.
- Llamamos recursivamente a la función `insert_helper` en el hijo correspondiente.


##### **(B) Insertar la nueva clave en la hoja**

Insertamos la nueva clave en la hoja. Este proceso se implementa en la función `insert_key_into_list`. La idea es insertar `new_key` al final y seguir intercambiando claves consecutivas hasta que la lista vuelva a estar ordenada.

- Si como resultado el nodo tiene $\leq 2d$ claves, hemos terminado.
- Si, en cambio, el nodo se llena (es decir, ahora tiene $2d+1$ claves), hay que corregirlo.

Para corregir un nodo lleno, lo dividimos en tres partes:
- La clave mediana `keys[d]`
- La lista de claves a la izquierda de la mediana
- La lista de claves a la derecha de la mediana

Para ello, creamos dos nodos (en realidad, reutilizamos el nodo actual como uno de los dos), uno que contendrá las claves a la izquierda y otro con las claves a la derecha. Esto se implementa en la función `split_node_into_two`.

Como resultado, necesitamos insertar la clave mediana en el nodo padre del nodo actual, con los dos nuevos nodos como hijos izquierdo y derecho.

##### **(C) Insertar la clave en el nodo padre**

Para insertar una clave en el nodo padre, usamos los valores de retorno de la función `insert_helper`. Cuando un nodo padre llama a `insert_helper` en su hijo número $i$, el valor retornado puede ser `None`, indicando que la inserción ha terminado, o una tripleta `(median_key, left, right)`, que nos indica que debemos insertar `median_key` en el nodo actual, con `left` y `right` como hijos izquierdo y derecho, respectivamente.

De hecho, sabemos que `median_key` se insertará en la posición `i`, desplazando las demás claves una posición a la derecha. De forma similar, el nodo `left` se convertirá en el nuevo hijo número `i` y `right` se insertará como hijo número `i+1`. Los demás hijos se desplazarán una posición a la derecha.

Para manejar esto, utilizamos la función `insert_key_and_ptr`.

##### **(D) Nueva raíz**

En algunos casos, la inserción creará una nueva raíz para el B-Tree. Esto debe ser manejado por la función `insert`, que devolverá el puntero a la nueva raíz si el nodo raíz ha cambiado, o simplemente el nodo raíz original si no ha cambiado.

In [ ]:
class BTreeNodeWithInsert(BTreeNodeBase):
    
    def __init__(self, keys = [], ptrs = [], is_root=False, d = 10):
        super().__init__(keys, ptrs, is_root, d)
        
    def create_new_instance(self, keys, ptrs, is_root, d):
        """Necesitamos esto para construir una nueva instancia porque decidimos dividir la implementación en tres clases diferentes"""
        return BTreeNodeWithInsert(keys, ptrs, is_root, d)
    
    def insert(self, new_key):
        """Inserta una nueva clave new_key en el árbol. 
           Llama a esta función solo si self es el nodo raíz"""
        assert self.is_root
        res = self.insert_helper(new_key)  # la función auxiliar contiene la lógica de inserción
        if res != None:
            (mid_key, n1, n2) = res  # si la función auxiliar retorna una tripleta (clave media y dos nuevos nodos)
            self.is_root = False  # necesitamos crear una nueva raíz
            new_root = self.create_new_instance([mid_key], [n1, n2], True, self.d)  # crear una nueva raíz con una clave y dos hijos
            n1.set_parent(new_root, 0)  # establecer los punteros al padre para n1 y n2
            n2.set_parent(new_root, 1)
            return new_root  # retornar la nueva raíz
        else:
            return self  # de lo contrario, la raíz original no cambió

    def insert_helper(self, new_key, debug=True):
        """Esta es una función auxiliar para insertar una nueva clave new_key en un nodo.
        Retorna None si hay espacio para la clave,
        o una tripleta (clave_media, n1, n2) para ser insertada en el nodo padre."""
        # si el nodo es una hoja
        if self.is_leaf():
            self.insert_key_into_list(new_key)  # insertar la clave en la lista
            n = len(self.keys)  # contar el número de claves
            if n <= 2 * self.d:  # el tamaño del nodo es aceptable
                return None  # hemos terminado
            else:
                # el nodo está lleno, se necesita dividir
                assert n == 2 * self.d + 1  # el nodo se llenó debido a esta nueva inserción
                (mid_key, n1, n2) = self.split_node_into_two()  # dividirlo en dos nodos
                return (mid_key, n1, n2)  # retornar la clave media y los dos nodos

        else:
            # encontrar el primer índice i tal que self.keys[i] >= new_key
            i = 0
            n = len(self.keys)
            while i < n and self.keys[i] < new_key:
                i = i + 1
            # no deberíamos encontrar una copia de la clave
            if i < n and self.keys[i] == new_key:
                if debug:
                    print(f'La clave {new_key} ya existe')  # esto no debería pasar, pero lo ignoramos por ahora
                return None
            else:
                res = self.pointers[i].insert_helper(new_key)  # insertar en el hijo correspondiente
                if res != None:
                    (mid_key, node_left, node_right) = res  # desempaquetar
                    # insertar la nueva clave proveniente del hijo en self junto con los dos punteros
                    self.insert_key_and_ptr(mid_key, node_left, node_right, i)
                    # ¿el nodo se llenó debido a la inserción?
                    if len(self.keys) == 2 * self.d + 1:
                        (mid_key, n1, n2) = self.split_node_into_two()  # dividir el nodo actual en dos
                        return (mid_key, n1, n2)  # retornar la clave media y los nodos al padre
                 
    def insert_key_into_list(self, new_key):
        """Insertar new_key en la lista de claves de este nodo. 
           Llamar a esta función solo en nodos hoja"""
        assert self.is_leaf()
        n = len(self.keys)
        assert new_key not in self.keys, f'La clave {new_key} ya existe {self.keys}'
        self.keys.append(new_key)
        i = n
        while i >= 1 and self.keys[i] < self.keys[i-1]:
            # intercambiar
            (self.keys[i-1], self.keys[i]) = (self.keys[i], self.keys[i-1])
            i = i - 1
            
    def insert_key_and_ptr(self, mid_key, node_left, node_right, i):
        """Insertar la clave mid_key en la posición i de la lista de claves.
           Asegurar que su hijo izquierdo sea node_left y su hijo derecho sea node_right."""
        n = len(self.keys)
        assert i >= 0 and i <= n
        node_left.set_parent(self, i)
        assert self.pointers[i] == node_left
        (new_key, new_child) = (mid_key, node_right)
        for j in range(i, n):
            (self.keys[j], new_key) = (new_key, self.keys[j])
            (self.pointers[j+1], new_child) = (new_child, self.pointers[j+1])
            self.pointers[j+1].set_parent(self, j+1)  # actualizar el puntero al padre, ya que la posición cambió
        self.keys.append(new_key)
        self.pointers.append(new_child)
        new_child.set_parent(self, n+1)
        
    def fix_parent_pointers_for_children(self):
        for (j, child_node) in enumerate(self.pointers):
            child_node.set_parent(self, j)
        
    def split_node_into_two(self):
        """Dividir un nodo en dos usando la clave mediana. 
           Llamar solo si el nodo está lleno"""
        assert len(self.keys) == 2 * self.d + 1  # el nodo está lleno
        n = len(self.keys)
        d = self.d
        med_key = self.keys[d]
        new_keys = list(self.keys[d+1:])  # tomar todas las claves después de la mediana
        self.keys = list(self.keys[:d])
        if self.is_leaf():
            new_ptrs = []
        else:
            new_ptrs = list(self.pointers[d+1:])
            self.pointers = list(self.pointers[:d+1])
        new_node = self.create_new_instance(new_keys, new_ptrs, False, d)  # crear nuevo nodo
        new_node.fix_parent_pointers_for_children()  # asegurarse de actualizar los punteros al padre para los hijos del nuevo nodo
        return (med_key, self, new_node)  # retornar la clave media y los dos nodos resultantes


In [ ]:
%matplotlib inline
import networkx as nx 
from matplotlib import pyplot as plt 

def draw_btree_graph(n):
    G = nx.DiGraph()
    labels = {}
    n.make_networkx_graph(G, 0, -1, labels)  # construir el grafo a partir del árbol B
    pos = nx.nx_agraph.graphviz_layout(G, prog="dot")  # obtener las posiciones para todos los nodos usando layout tipo jerárquico
    fig, ax = plt.subplots()
    fig.set_tight_layout(True)
    
    # Dibujar el grafo con etiquetas, nodos en forma de cuadrados, tamaño de fuente pequeño, sin color de relleno, con bordes
    nx.draw(
        G,
        pos=pos,
        with_labels=True,
        node_shape="s",
        font_size=8,
        labels=labels,
        node_color="none",
        bbox=dict(facecolor="cyan", edgecolor='black')
    )
    
    # Establecer márgenes para los ejes para que los nodos no se corten
    ax = plt.gca()
    ax.margins(0.20)
    plt.axis("off")  # ocultar los ejes
    plt.show() 

##### **Ejemplo 1**

Considera el siguiente B-Tree con $d=2$.

In [ ]:
lst = [1, 5, 2, 4, 3, 9, 15, -5, 12, 18, 80, -25, 22, 31, -15]
b = BTreeNodeWithInsert(d=2,is_root=True)
for k in lst:
    b = b.insert(k)
b.rep_ok()
draw_btree_graph(b)

Supongamos que deseamos insertar una nueva clave `-2` en el árbol.  
  - La primera fase desciende desde la raíz hasta la hoja `C1`, ya que $-5 \leq -2 \leq 3$.  
  - La inserción ocurre en el hijo `C1`, que ahora contiene $[-2, 1, 2]$.  
  - Como el hijo `C1` no está lleno, no se requiere ninguna acción adicional.

In [ ]:
b = b.insert(-2)
b.rep_ok()
draw_btree_graph(b)

Supongamos que queremos insertar $71$, ¿qué ocurriría?

In [ ]:
b = b.insert(71)
b.rep_ok()
draw_btree_graph(b)

Ahora intentemos insertar `29`. Esto provocará una inserción en `C4`.  
  - Sin embargo, `C4` estará lleno después de la inserción. Lo dividiremos en dos nodos e insertaremos la clave mediana `31` en la raíz.  
  - La raíz ahora estará llena y se dividirá en dos nodos con el valor mediano $9$.  
  - Finalmente, tendremos una nueva raíz con solo una clave: $9$.

In [ ]:
b = b.insert(29)
b.rep_ok()
draw_btree_graph(b)

In [ ]:
b.insert(16)
draw_btree_graph(b)

#### Eliminando una clave

Finalmente, crearemos una clase **BTreeNode** que extienda las capacidades de inserción previamente implementadas, añadiendo la posibilidad de eliminar una clave del árbol.

##### **(A) Mover la eliminación a un nodo hoja**

- **Localización de la clave:**  
  Para eliminar una clave, primero se localiza en el árbol utilizando el método `find_internal`, el cual devuelve el nodo y el índice en dicho nodo donde se encuentra la clave.

- **Caso de nodo no-hoja:**  
  Si el nodo es no-hoja, se reemplaza la clave a eliminar por su sucesor.  
  El sucesor se halla tomando el hijo derecho inmediato y, a continuación, desplazándose hasta la hoja más a la izquierda de ese hijo.

*Ejemplo:*  
Si se desea eliminar la clave `9` que se encuentra en la raíz (nodo no-hoja), se busca el sucesor (la siguiente clave mayor en el árbol), que resulta ser, por ejemplo, `12`. Primero se reemplaza la clave `9` por `12` y, posteriormente, se elimina la clave `12` de la hoja. Eliminar `12` de la hoja no viola ninguna restricción si el tamaño del nodo hoja sigue siendo, por ejemplo, 2.

Esta lógica se implementa en la función `delete_key_helper`.

##### **(B) Manejar los nodos que quedan "subllenos"**

Si, en el proceso anterior, el nodo hoja queda con un número de claves menor que el umbral $d$, se deben considerar tres casos:

1. **Préstamo desde el hermano derecho:**  
   Si el nodo tiene un hermano a la derecha con $d+1$ o más claves, se puede "tomar prestada" una clave desde dicho hermano.

2. **Préstamo desde el hermano izquierdo:**  
   Si no se cumple el caso anterior y el nodo tiene un hermano a la izquierda con $d+1$ o más claves, se toma una clave prestada desde ese hermano.

3. **Fusión con un hermano lleno:**  
   Si ninguna de las opciones anteriores es aplicable, se fusiona el nodo sublleno con uno de sus hermanos "llenos" y se actualiza el nodo padre en consecuencia.

Esta lógica se implementa en la función `fix_underfull_node`.



##### **(B.1) Tomar prestada una clave de un nodo hermano**

Para solucionar una situación en la que un nodo está sublleno tomando prestada una clave del hermano derecho, se procede de la siguiente manera:

- Se mueve la clave $k$ del nodo padre hacia el nodo actual.
- Se mueve la clave más a la izquierda, $k'$, del hermano derecho hacia el nodo padre.
- Se transfiere el puntero más a la izquierda del nodo hermano derecho para que se convierta en el puntero más a la derecha del nodo actual.

Esto se implementa en la función `borrow_from_right_sibling`. Como resultado, el nodo actual terminará con $d$ claves y el nodo hermano tendrá una clave menos de lo que tenía anteriormente. La situación es análoga para el caso de tomar prestada una clave del hermano izquierdo (en caso de existir), implementada en la función `borrow_from_left_sibling`.

##### **(B.2) Fusionar con un nodo hermano**

Si el nodo hermano cuenta también con $d$ claves, tomar una clave prestada no es viable, ya que ello haría que el nodo hermano se vuelva sublleno. En ese caso se procede a fusionar dos nodos en uno:

- El nodo actual contiene $d-1$ claves.
- El nodo hermano contiene $d$ claves.
- Se mueve una clave desde el nodo padre hacia abajo, uniéndola a la fusión.

El resultado de la fusión es un nodo con $2d$ claves. Este proceso requiere además eliminar la clave del nodo padre, lo que podría provocar que el propio nodo padre se vuelva sublleno y deba ser corregido posteriormente, a menos que sea la raíz.

La función  `merge_with_right_sibling`  implementa este proceso.

##### **(B.3) La raíz puede quedar vacía**

Durante este proceso, es posible que la raíz quede vacía. Esto solo puede suceder si:
  - La raíz tenía previamente una única clave y dos hijos.
  - Los dos hijos de la raíz se fusionan, eliminando la única clave en la raíz.

Por lo tanto, necesitaremos manejar este caso eliminando la raíz y haciendo que el único hijo restante se convierta en la nueva raíz.


In [ ]:
class BTreeNode(BTreeNodeWithInsert):
    
    def __init__(self, keys = [], ptrs = [], is_root=False, d = 10):
        super().__init__(keys, ptrs, is_root, d)
    
    def create_new_instance(self, keys, ptrs, is_root, d):
        """Necesitamos esto para construir una nueva instancia porque decidimos dividir la implementación en tres clases diferentes"""
        return BTreeNode(keys, ptrs, is_root, d)
    
    def delete_key(self, key_to_delete):
        """Esta es la rutina principal para eliminar una clave del árbol. Debe ser llamada desde la raíz del árbol."""
        assert self.is_root  # llamar a esta rutina solo desde la raíz
        result = self.delete_key_helper(key_to_delete)  # llamar al método auxiliar
        if result == None:
            # la raíz no cambió
            return self
        else:
            return result  # el resultado es el nuevo nodo raíz del árbol
        
    def delete_key_helper(self, key_to_delete, debug=True):
        """Este es el método principal de trabajo para eliminar una clave del árbol."""
        # primero encontramos el nodo en el árbol que contiene la clave
        res = self.find_key_internal(key_to_delete)
        if res == None:  # la clave no fue encontrada, no hay nada que eliminar
            print(f'La clave {key_to_delete} no existe en el árbol')  # advertencia para el usuario
            return None  # salir
        # De lo contrario, encontramos la clave y debemos eliminarla
        (node, idx) = res  # desempaquetar el resultado de find_key_internal para obtener el nodo e índice
        assert 0 <= idx and idx < len(node.keys)  # el índice debe ser válido
        assert node.keys[idx] == key_to_delete  # si esto falla, el método de búsqueda está incorrecto
        if not node.is_leaf():  # si no es un nodo hoja
            (succ_key, successor_leaf_node) = node.find_successor(idx)  # encontrar la clave sucesora y su nodo
            assert successor_leaf_node.is_leaf()
            assert succ_key > node.keys[idx], f"Sucesor: {succ_key}, clave en nodo {node.keys[idx]}"
            assert succ_key == successor_leaf_node.keys[0]
            # reemplazar la clave con la del sucesor
            if debug:
                print(f'Reemplazando {key_to_delete} con la clave sucesora {succ_key}')
            node.keys[idx] = succ_key  # reemplazar la clave
            return successor_leaf_node.delete_key_leaf(0, debug)  # eliminar la clave sucesora desde la hoja
        else:  # ya estamos en una hoja
            return node.delete_key_leaf(idx, debug)  # eliminar la clave desde la hoja
            
    def delete_key_leaf(self, idx, debug=True):
        """Eliminar una clave que está en una hoja. self es un nodo hoja e idx es el índice a eliminar."""
        assert self.is_leaf()  # no llamar este método en nodos no hoja
        n = len(self.keys)
        d = self.d
        assert 0 <= idx and idx < n
        for i in range(idx, n-1):
            self.keys[i] = self.keys[i+1]  # mover claves a la izquierda, sobrescribiendo la clave en idx
        self.keys.pop()  # eliminar el último elemento
        assert len(self.keys) == n-1  # debe haber una clave menos
        if self.is_root or (n-1) >= d:  # verificar si está por debajo del mínimo
            return None  # no está por debajo — no hay nada más que hacer
        else:
            # tenemos un nodo por debajo del mínimo
            print(f'La eliminación causa que el nodo con claves {self.keys} quede por debajo del mínimo')
            return self.fix_underfull_node(debug)  # corregir el nodo

    def fix_underfull_node(self, debug=True):
        """Corregir un nodo que ha quedado por debajo del mínimo después de eliminar una clave."""
        if self.is_root:  # si ya estamos en la raíz, no hay nada que corregir
            return None
        if len(self.keys) >= self.d:  # verificar que efectivamente esté por debajo del mínimo
            return None
        # el nodo realmente está por debajo
        assert len(self.keys) == self.d - 1
        n = len(self.keys)
        d = self.d
        assert self.parent != None  # no es la raíz, debe tener un nodo padre
        (parent_node, node_idx) = self.parent  # obtener el nodo padre y su índice
        n_parent = len(parent_node.keys)  # cuántas claves tiene el padre
        assert n_parent >= 1  # el padre debe tener al menos una clave
        if node_idx <= n_parent - 1:
            # tengo un hermano a la derecha
            right_sibling_node = parent_node.pointers[node_idx+1]
            n_right_sibling = len(right_sibling_node.keys)
            if n_right_sibling > d:
                if debug:
                    print('Pidiendo prestada una clave del hermano derecho')
                self.borrow_from_right_sibling(right_sibling_node)
            else:
                if debug:
                    print('Fusionando con el hermano derecho')
                self.merge_with_right_sibling(right_sibling_node)
                if parent_node.is_root and len(parent_node.keys) == 0:
                    if debug:
                        print('La raíz quedó vacía: designando nueva raíz.')
                    self.parent = None
                    self.is_root = True
                    self.fix_parent_pointers_for_children()
                    return self
        else:
            assert node_idx >= 1
            # tengo un hermano a la izquierda
            left_sibling_node = parent_node.pointers[node_idx-1]
            n_left_sibling = len(left_sibling_node.keys)
            if n_left_sibling > d:
                if debug:
                    print('Pidiendo prestada una clave del hermano izquierdo')
                self.borrow_from_left_sibling(left_sibling_node)
            else:
                if debug:
                    print('Fusionando con el hermano izquierdo')
                left_sibling_node.merge_with_right_sibling(self)
                if parent_node.is_root and len(parent_node.keys) == 0:
                    if debug:
                        print('La raíz quedó vacía: designando nueva raíz.')
                    left_sibling_node.parent = None
                    left_sibling_node.is_root = True
                    left_sibling_node.fix_parent_pointers_for_children()
                    return left_sibling_node

        # si el padre no es la raíz y ha quedado por debajo
        if not parent_node.is_root and len(parent_node.keys) < d:
            # el nodo padre está por debajo, corregir recursivamente
            return parent_node.fix_underfull_node()

    def borrow_from_right_sibling(self, right_sibling_node):
        """Pedir prestada una clave del hermano derecho. Este no debe quedar por debajo del mínimo tras el préstamo."""
        assert self.parent != None
        (parent_node, parent_idx) = self.parent  # obtener el padre
        assert right_sibling_node.parent != None
        assert right_sibling_node.parent[0] == parent_node
        assert right_sibling_node.parent[1] == parent_idx+1
        n_right_sibling = len(right_sibling_node.keys)
        assert n_right_sibling >= self.d + 1
        # Pedir clave prestada
        self.keys.append(parent_node.keys[parent_idx])  # añadir la clave del padre a este nodo
        parent_node.keys[parent_idx] = right_sibling_node.keys[0]  # reemplazar clave del padre con la primera del hermano derecho
        right_sibling_node.keys.pop(0)  # eliminar la clave del hermano
        if not self.is_leaf():  # si no es hoja, también hay que mover punteros
            assert not right_sibling_node.is_leaf()
            new_child_node = right_sibling_node.pointers[0]  # obtener el primer hijo del hermano
            self.pointers.append(new_child_node)  # añadirlo como último hijo
            right_sibling_node.pointers.pop(0)
            self.fix_parent_pointers_for_children()
            right_sibling_node.fix_parent_pointers_for_children()
        return

    def borrow_from_left_sibling(self, left_sibling_node):
        """Pedir prestada una clave del hermano izquierdo. Mismo proceso que con el hermano derecho."""
        assert self.parent != None
        (parent_node, parent_idx) = self.parent
        assert left_sibling_node.parent != None
        assert left_sibling_node.parent[0] == parent_node
        assert left_sibling_node.parent[1] == parent_idx - 1
        n_sibling = len(left_sibling_node.keys)
        assert n_sibling >= self.d + 1
        self.keys.insert(0, parent_node.keys[parent_idx - 1])  # insertar clave del padre como primera
        parent_node.keys[parent_idx - 1] = left_sibling_node.keys[-1]  # reemplazar en padre con la última del hermano
        left_sibling_node.keys.pop()  # eliminar la última del hermano izquierdo
        if not self.is_leaf():
            assert not left_sibling_node.is_leaf()
            self.pointers.insert(0, left_sibling_node.pointers[n_sibling])  # mover el puntero derecho del hermano como primero
            left_sibling_node.pointers.pop()
            self.fix_parent_pointers_for_children()
            left_sibling_node.fix_parent_pointers_for_children()
        return

    def merge_with_right_sibling(self, right_sibling_node):
        """Fusionar con el hermano derecho en un solo nodo. Implica eliminar una clave del nodo padre."""
        assert self.parent != None
        (parent_node, parent_idx) = self.parent
        assert right_sibling_node.parent != None
        assert right_sibling_node.parent[0] == parent_node
        assert right_sibling_node.parent[1] == parent_idx + 1
        self.keys = self.keys + [parent_node.keys[parent_idx]] + right_sibling_node.keys  # fusionar claves y añadir la del padre
        assert len(self.keys) == 2 * self.d
        if not self.is_leaf():
            assert not right_sibling_node.is_leaf()
            self.pointers = self.pointers + right_sibling_node.pointers  # fusionar punteros
            assert len(self.pointers) == 2 * self.d + 1, f'Número de punteros = {len(self.pointers)}, d = {self.d}'
        parent_node.keys.pop(parent_idx)  # eliminar clave del padre
        parent_node.pointers.pop(parent_idx + 1)  # eliminar puntero al hermano derecho
        parent_node.fix_parent_pointers_for_children()  # corregir punteros del padre
        if not self.is_leaf():
            self.fix_parent_pointers_for_children()  # corregir mis propios punteros
        return


##### **Ejemplo**

Considera el ejemplo siguiente.

In [ ]:
lst = [1, 5, 2, 4, 3, 9, 15, -5, 12, 18, 80, -25, 22, 31, -15, -2, 71, 29, 16]
b = BTreeNode(d=2,is_root=True)
for k in lst:
    b = b.insert(k)
b.rep_ok()
draw_btree_graph(b)


Si intentamos eliminar la clave `9`, se reemplaza por su sucesor `12`.

In [ ]:
b = b.delete_key(9)
b.rep_ok()
draw_btree_graph(b)

Supongamos que ahora intentamos eliminar la clave $22$. Esto hará que el nodo 

$$
R \rightarrow C1 \rightarrow C1
$$

quede con un número insuficiente de claves. Dado que el nodo no puede tomar prestado a su hermano, necesita fusionarse con uno. Esto, a su vez, elimina una clave de $R \rightarrow C1$ y provoca una fusión entre la raíz anterior, $C0$ y $C1$. La raíz ahora está vacía y debe ser reemplazada por una nueva raíz.

In [ ]:
b = b.delete_key(22)
b.rep_ok()
draw_btree_graph(b)

Supongamos que queremos eliminar la clave $12$ en la raíz; esta es reemplazada por la clave $15$ proveniente de $C3$. $C3$ queda con menos claves de las necesarias y, para solucionarlo, tomamos prestada una clave de $C4$.

In [ ]:
b = b.delete_key(12)
b.rep_ok()
draw_btree_graph(b)

In [ ]:
from random import shuffle
l = list(range(-40,40))
shuffle(l)
print(l)
b = BTreeNode(d=3,is_root=True)
for j in l:
    b = b.insert(j)
b.rep_ok()
draw_btree_graph(b)

In [ ]:
from random import shuffle
l = list(range(-500,500))
shuffle(l)
print(l)
b = BTreeNode(d=7,is_root=True)
for j in l:
    b = b.insert(j)
b.rep_ok()
display(draw_btree_graph(b))
for i in range(-425, 420):
    if b.find_key(i):
        b = b.delete_key(i)
b.rep_ok()
draw_btree_graph(b)

#### **Ejercicios**

##### **Ejercicio 1: simulación manual de inserciones y divisiones**

Revisa paso a paso cómo se construye el B-Tree a partir de una secuencia de inserciones y comprender el momento en el que se producen las divisiones.

**Instrucciones:**  
1. Toma la siguiente secuencia de claves:  
   ```
   [1, 5, 2, 4, 3, 9, 15, -5, 12, 18, 80, -25, 22, 31, -15]
   ```
2. Con un parámetro $ d = 2 $ para los nodos, dibuja manualmente (en papel o usando alguna herramienta gráfica) el árbol después de cada inserción.  
3. Presta atención a:
   - En qué momento un nodo hoja se llena (pasa a tener $2d + 1$ claves).
   - Cómo se realiza la división del nodo (identifica la clave mediana, el nodo izquierdo y el nodo derecho).
   - Cómo se propaga la división al nodo padre, e incluso a la raíz.
4. Finalmente, explica por qué la raíz puede llegar a cambiar y de qué manera.

**Puntos a evaluar:**  
- Detección correcta del punto de división.  
- Comprensión de la inserción en el nodo padre mediante el método `insert_key_and_ptr`.

##### **Ejercicio 2: Seguimiento del proceso de búsqueda y eliminación**

Analiza cómo se localiza una clave en el árbol y qué pasos se siguen para su eliminación, especialmente cuando la clave se encuentra en un nodo no hoja.

**Instrucciones:**  
1. Con la estructura resultante del Ejercicio 1 (o utilizando el B-Tree ya construido con el código), selecciona la clave `9` para eliminar.
2. Simula manualmente el proceso:
   - Usa el método `find_key_internal` para identificar en qué nodo se encuentra la clave.
   - Si la clave está en un nodo no hoja, explica cómo se obtiene el sucesor (usando `find_successor`) y cuál es el procedimiento para reemplazar y luego eliminar la clave sucesora en la hoja.
3. Dibuja el árbol antes y después de la eliminación de la clave `9` y comenta cómo se han mantenido las propiedades del B-Tree (por ejemplo, que todas las hojas sigan estando a la misma altura y que los nodos tengan el número mínimo de claves).

**Puntos a evaluar:**  
- Comprensión del método `find_key_internal`.  
- Correcta identificación y explicación del proceso de reemplazo y eliminación en nodos no hoja.


##### **Ejercicio 3: Manejo de nodos subllenos tras la eliminación**

Estudia y practica las operaciones de reequilibrado (préstamo y fusión) cuando, tras la eliminación de una clave, algún nodo queda por debajo del umbral mínimo.

**Instrucciones:**  
1. Utiliza el B-Tree generado previamente y elimina la clave `22`. Se indica en el ejemplo que esto causa que un nodo (por ejemplo, en la ruta $R \rightarrow C1 \rightarrow C1$) quede con claves insuficientes.
2. Sigue manualmente el proceso que lleva a:
   - No poder pedir una clave prestada (verifica que los hermanos no tengan claves de más).
   - Realizar la fusión de nodos mediante el método `merge_with_right_sibling` o `merge_with_left_sibling`.
   - Propagar esta fusión al nodo padre y, si es necesario, hacer que la raíz se actualice.
3. Dibuja el árbol antes y después de la eliminación de `22` y detalla cada paso del proceso de corrección.

**Puntos a evaluar:**  
- Comprensión del algoritmo de corrección de nodos subllenos (método `fix_underfull_node`).  
- Capacidad para identificar cuándo se debe pedir prestada una clave y cuándo se debe fusionar nodos.

##### **Ejercicio 4: Extensión de permitir duplicados en el B-Tree**
  
Modifica la implementación para aceptar claves duplicadas, si se desea, y analizar las implicaciones en las operaciones de inserción, búsqueda y eliminación.

**Instrucciones:**  
1. Investiga en la literatura o en la presentación original (por ejemplo, en la revisión de Douglas Comer) cómo se podría manejar la inserción de claves duplicadas en un B-Tree.
2. Propón al menos dos estrategias (por ejemplo, almacenar una lista de valores asociados a cada clave o ajustar la comparación de claves de modo que se permita la repetición).
3. Realiza las modificaciones necesarias en los métodos `insert_key_into_list`, `find_key_internal` y `delete_key_helper` para manejar duplicados.
4. Implementa casos de prueba donde insertes claves duplicadas y verifica (usando `rep_ok()`) que el árbol mantiene sus invariantes.

**Puntos a evaluar:**  
- Diseño de una estrategia para gestionar duplicados.  
- Implementación y pruebas que demuestren la integridad del árbol con la nueva funcionalidad.

##### **Ejercicio 5: Implementación de una función de recorrido en orden**

Agrega un método al B-Tree que recorra (traverse) todas las claves en orden ascendente, utilizando recursión y aprovechando la estructura de los nodos.

**Instrucciones:**  
1. Define una función `in_order_traversal()` que recorra el árbol. La función debe retornar una lista con todas las claves ordenadas.
2. Recuerda que en un nodo de B-Tree, para cada clave `keys[i]`:
   - El subárbol apuntado por `pointers[i]` contiene claves menores a `keys[i]`.
   - El subárbol apuntado por `pointers[i+1]` contiene claves mayores que `keys[i]`.
3. Prueba tu función con el B-Tree actual (por ejemplo, tras varias inserciones y eliminaciones) y compara la lista resultante con la lista de claves ordenada obtenida con la función nativa de Python `sorted()`.
4. Documenta y comenta cada paso para facilitar la comprensión.

**Puntos a evaluar:**  
- Correcta implementación del recorrido en orden para estructuras de árbol B.  
- Comparación de resultados y validación de la correcta secuencia ascendente.


##### **Ejercicio 6: Pruebas de estrés y análisis de rendimiento**

Realiza pruebas de inserción y eliminación de un gran número de claves para evaluar la robustez de la implementación y observar la estructura visualmente en diferentes escenarios.

**Instrucciones:**  
1. Usa el siguiente fragmento de código modificado para trabajar con un rango mayor de números (por ejemplo, de -500 a 500) y con un parámetro $ d $ mayor (por ejemplo, $ d = 7 $):
   ```python
   from random import shuffle
   l = list(range(-500, 500))
   shuffle(l)
   b = BTreeNode(d=7, is_root=True)
   for key in l:
       b = b.insert(key)
   b.rep_ok()
   draw_btree_graph(b)
   ```
2. Realiza eliminaciones masivas sobre el árbol (por ejemplo, elimina todas las claves en un rango determinado).
3. Mide el tiempo de ejecución de un bloque de inserciones y eliminaciones utilizando el módulo `time` de Python.
4. Analiza:
   - Cómo cambia la altura del árbol conforme aumenta el número de claves.
   - Si se mantienen las propiedades estructurales después de cada operación.
   - La eficiencia en tiempo de búsqueda y reestructuración.
5. Documenta tus observaciones y plantea conclusiones sobre el comportamiento del B-Tree en condiciones de "estrés".

**Puntos a evaluar:**  
- Ejecución correcta de inserciones y eliminaciones masivas.  
- Análisis del rendimiento (por ejemplo, altura del árbol, tiempos de ejecución, etc.).  
- Observaciones fundamentadas sobre la eficiencia y robustez de la estructura.


In [ ]:
### Tus respuestas